In [ ]:
using DifferentialEquations
using Plots
using Interact
using ODEInterfaceDiffEq
using SimpleDiffEq
using BenchmarkTools

# Van der Pol model

We consider the following problem :

$$
\left\{ 
\begin{aligned} 
{\mathrm d}_t y_1 & = \varepsilon^2 \left(y_2-\frac{y_1^3}{3}+y_1 \right) \quad{} \text{with } \varepsilon > 0\\ 
{\mathrm d}_t y_2 & = - y_1 
\end{aligned} 
\right. 
$$

In [ ]:
function vanderpol!(du, u, p, t)
  eps = p
  du[1] = eps^2 * (u[2] - (u[1]^3)/3 + u[1])
  du[2] = -u[1]
end;

## Quasi exact solution

The quasi-exact solution is obtained by using an implicit Runge-Kutta method of Radau IIA family of order 5 with stepsize control and fine tolerances.

In [ ]:
u0 = [0.5; 0.0]
tspan = (0.0, 10.0)

a = @manipulate for eps=slider(1:1:20, value=1, label="eps")
    prob = ODEProblem(vanderpol!, u0, tspan, eps)
    sol = solve(prob, RadauIIA5(), abstol=1e-10,  reltol=1e-10)
    plot(sol, vars=1, size=(900,300), xaxis = ("t"), leg=false, margin=5mm, 
         linewidth=2, title="Quasi exact solution : y1", titlefontsize=12)
end
@layout! a dom"div"( observe(_), vbox(:eps))

## Radau5 integration

In [ ]:
u0 = [0.5; 0.0]
tspan = (0.0, 10.0)
eps = 10.0
tol = 1e-4

prob = ODEProblem(vanderpol!, u0, tspan, eps)
sol = solve(prob, RadauIIA5(), abstol=tol,  reltol=tol)
sol_exa = solve(prob, RadauIIA5(), saveat=sol.t, abstol=1e-12, reltol=1e-12)

err_y1 = abs.(sol[1,:] - sol_exa[1,:])

println("Radau5 integration with tol = ", tol)
print("Elapsed Time : ")
@btime sol = solve(prob, RadauIIA5(), abstol=tol,  reltol=tol)
println("Number of function evaluations : ", sol.destats.nf)
println("Number of linear solves :        ", sol.destats.nsolve)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

ps = plot(sol, vars=1, seriestype=:scatter, xaxis = ("t"), leg=false, size=(900,600), margin=5mm, 
          title="Radau5 solution : y1", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(sol.t, err_y1, seriestype=:scatter, xaxis = ("t"), leg=false, size=(900,600), margin=5mm, 
          title="Global error : |y1 - y1exa|", titlefontsize=12, markershape=:xcross, markersize=2)
plot(ps, pe, layout=(2,1))

## Dopri 5 integration

In [ ]:
u0 = [0.5; 0.0]
tspan = (0.0, 10.0)
eps = 10.0
tol = 1e-4

prob = ODEProblem(vanderpol!, u0, tspan, eps)
sol = solve(prob, dopri5(), abstol=tol,  reltol=tol)
sol_exa = solve(prob, RadauIIA5(), saveat=sol.t, abstol=1e-12, reltol=1e-12)

err_y1 = abs.(sol[1,:] - sol_exa[1,:])

println("Dopri5 integration with tol = ", tol)
print("Elapsed Time : ")
@btime sol = solve(prob, dopri5(), abstol=tol,  reltol=tol)
println("Number of function evaluations : ", sol.destats.nf)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

ps = plot(sol, vars=1, seriestype=:scatter, xaxis = ("t"), leg=false, size=(900,600), margin=5mm, 
          title="Dopri5 solution : y1", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(sol.t, err_y1, seriestype=:scatter, xaxis = ("t"), leg=false, size=(900,600), margin=5mm, 
          title="Global error : |y1 - y1exa|", titlefontsize=12, markershape=:xcross, markersize=2)
plot(ps, pe, layout=(2,1))

## Rock4 integration

In [ ]:
u0 = [0.5; 0.0]
tspan = (0.0, 10.0)
eps = 10.0
tol = 1e-4

prob = ODEProblem(vanderpol!, u0, tspan, eps)
sol = solve(prob, ROCK4(), abstol=tol,  reltol=tol)
sol_exa = solve(prob, RadauIIA5(), saveat=sol.t, abstol=1e-12, reltol=1e-12)

err_y1 = abs.(sol[1,:] - sol_exa[1,:])

println("ROCK4 integration with tol = ", tol)
print("Elapsed Time : ")
@btime sol = solve(prob, ROCK4(), abstol=tol,  reltol=tol)
println("Number of function evaluations : ", sol.destats.nf)
println("Number of accepted steps :       ", sol.destats.naccept)
println("Number of rejected steps :       ", sol.destats.nreject)

ps = plot(sol, vars=1, seriestype=:scatter, xaxis = ("t"), leg=false, size=(900,600), margin=5mm, 
          title="Dopri5 solution : y1", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(sol.t, err_y1, seriestype=:scatter, xaxis = ("t"), leg=false, size=(900,600), margin=5mm, 
          title="Global error : |y1 - y1exa|", titlefontsize=12, markershape=:xcross, markersize=2)
plot(ps, pe, layout=(2,1))

## A fixed timestep Runge Kutta 4

In [ ]:
u0 = [0.5; 0.0]
tspan = (0.0, 10.0)
eps = 1.0
dt = 5e-2

prob = ODEProblem(vanderpol!, u0, tspan, eps)
sol = solve(prob, LoopRK4(), dt=dt);
sol_exa = solve(prob, RadauIIA5(), saveat=sol.t, abstol=1e-12, reltol=1e-12)

err_y1 = abs.(sol[1,:] - sol_exa[1,:])

println("RK4 integration with dt= ", dt)
print("Elapsed Time : ")
@btime sol = solve(prob, LoopRK4(), dt=dt)
println("Number of function evaluations : ", 4*(10/dt))

ps = plot(sol, vars=1, seriestype=:scatter, xaxis = ("t"), leg=false, size=(900,600), margin=5mm, 
          title="RK4 solution : y1", titlefontsize=12, markershape=:xcross, markersize=2)
pe = plot(sol.t, err_y1, seriestype=:scatter, xaxis = ("t"), leg=false, size=(900,600), margin=5mm, 
          title="Global error : |y1 - y1exa|", titlefontsize=12, markershape=:xcross, markersize=2)
plot(ps, pe, layout=(2,1))